In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv('combined_scrape_2.csv')

In [23]:
df.shape

(3543710, 4)

In [24]:
df.head()

,Comment Text,Author,Subreddit,Post ID
0,"Just curious, but what triggered your feelings...",PDXorax,YangForPresidentHQ,ew5ohva
1,Christmas time put up a light-up Yang sign!,PDXorax,YangForPresidentHQ,ew5o8vv
2,"During the general election, get in contact wi...",PDXorax,YangForPresidentHQ,ew42bdx
3,"We need to understand as a country, that the m...",PDXorax,YangForPresidentHQ,evva7l8
4,"Part of the reason for this, is because we nee...",PDXorax,YangForPresidentHQ,evoo179


In [25]:
df.dtypes

Comment Text    object
Author          object
Subreddit       object
Post ID         object
dtype: object

In [26]:
sum_series = df.groupby(['Author','Subreddit']).size()

In [27]:
users = list(set([x for x in sum_series.index.get_level_values(0)]))

In [28]:
len(users)

5292

In [29]:
political_dict={}
for user in users:
    #create target series
    inner_dict = {}
    try:
        inner_dict['yang'] = sum_series[user]['YangForPresidentHQ']
    except:
        inner_dict['yang'] = 0
    try:
        inner_dict['bernie'] = sum_series[user]['SandersForPresident']
    except:
        inner_dict['bernie'] = 0
    try:
        inner_dict['bernie'] += sum_series[user]['wayofthebern']
    except:
        inner_dict['bernie'] += 0
    try:
        inner_dict['bernie'] += sum_series[user]['Political_Revolution']
    except:
        inner_dict['bernie'] += 0
    try:
        inner_dict['pete'] = sum_series[user]['Pete_Buttigieg']
    except:
        inner_dict['pete'] = 0
    try:
        inner_dict['kamala'] = sum_series[user]['Kamala']
    except:
        inner_dict['kamala'] = 0
    try:
        inner_dict['biden'] = sum_series[user]['JoeBiden']
    except:
        inner_dict['biden'] = 0
    try:
        inner_dict['warren'] = sum_series[user]['ElizabethWarren']
    except:
        inner_dict['warren'] = 0
    try:
        inner_dict['tulsi'] = sum_series[user]['tulsi']
    except:
        inner_dict['tulsi'] = 0
    try:
        inner_dict['dem'] += sum_series[user]['centerleftpolitics']
    except:
        inner_dict['dem'] = 0
    try:
        inner_dict['dem'] += sum_series[user]['VoteBlue']
    except:
        inner_dict['dem'] += 0
    try:
        inner_dict['dem'] = sum_series[user]['BlueMidterm2018']
    except:
        inner_dict['dem'] = 0
    try:
        inner_dict['dem'] = sum_series[user]['liberal']
    except:
        inner_dict['dem'] = 0
    try:
        inner_dict['dem'] = sum_series[user]['democrats']
    except:
        inner_dict['dem'] = 0
    political_dict[user]=inner_dict


In [30]:
df_target = pd.DataFrame(political_dict).T

In [31]:
df_target.head(5)

,bernie,biden,dem,kamala,pete,tulsi,warren,yang
willbailes,19,0,0,0,0,0,0,0
PermanentMagnetMan,0,1,0,0,0,0,0,0
anonymousredditor0,8,1,0,0,0,0,0,0
rivalarrival,0,0,25,0,0,0,0,0
tehramz,0,0,17,0,0,0,0,0


In [32]:

##sensitivity analysis

In [33]:
def func(row):
    total = sum(row)
    if total < 5:
        return 'drop'
    if (row['yang'] > row['bernie']) & (row['yang'] > row['tulsi']) & (row['yang'] > row['warren']) & (row['yang'] > row['biden']) & (row['yang'] > row['kamala']) & (row['yang'] > row['pete']):
        return '1' #1 = yang supporter
    else:
        return '0' #0 = not yang supporter

In [34]:
# def func(row):
#     total = sum(row)
#     if total < 5:
#         return 'drop'
#     total_cand = row['bernie']+row['biden']+row['kamala']+row['pete']+row['yang']
#     if (row['yang']/total_cand >= .8) & (row['yang']>=4):
#         return '1' #1 = yang supporter
#     else:
#         return '0' #0 = not yang supporter
    

In [35]:
df_target['target'] = df_target.apply(func,axis=1)

In [36]:
df_target['target'].value_counts()

0       2380
drop    1975
1        937
Name: target, dtype: int64

In [37]:
df_target.drop(['bernie','biden','kamala','pete','yang','tulsi','warren','dem'],axis=1,inplace=True)

In [38]:
df_target = df_target[df_target['target']!='drop']

In [39]:
df_target.head()

,target
willbailes,0
anonymousredditor0,0
rivalarrival,0
tehramz,0
Choco_Bacon,1


In [40]:
y_train,y_test = train_test_split(df_target['target'],random_state=42,stratify = df_target['target'])

In [41]:
y_train.size

2487

In [42]:
y_test.size

830

In [43]:
pd.DataFrame(y_train).to_csv('fin_users_train.csv')

In [44]:
pd.DataFrame(y_test).to_csv('fin_users_test.csv')